In [15]:
import time
import numpy as np
from math import fabs
from random import normalvariate
from numba import int32, float32, vectorize, njit, guvectorize, float64, prange, intp  # import the types
from numba.experimental import jitclass

In [16]:
def gu_random_normal_weights(func, *args, **kwargs):
    """
    The guvectorize decorated method, runs in nopython mode
    :param func: a loss error method
    :param target: choose between | 1: None -> serial | 2: 'parallel' -> parallel | execution method
    :return: the decorated loss, error function
    """
    kwargs_ = {k: v for k, v in kwargs.items() if v is not None}
    return guvectorize([(float32, float32, float32[:,:], float32[:,:]),
              (float64, float64, float64[:,:], float64[:,:])],'(),(),(m,n)->(m,n)', nopython=True, fastmath=True, *args, **kwargs_)(func)

def random_normal_weights(mu, sigma, arr, out):
    dim1 = arr[0].shape
    dim2 = arr[1].shape
    for i in prange(*dim1):
        for j in prange(*dim2):
            out[i][j] = normalvariate(mu, sigma)

def gu_random_normal_bias(func, *args, **kwargs):
    """
    The guvectorize decorated method, runs in nopython mode
    :param func: a loss error method
    :param target: choose between | 1: None -> serial | 2: 'parallel' -> parallel | execution method
    :return: the decorated loss, error function
    """
    kwargs_ = {k: v for k, v in kwargs.items() if v is not None}
    return guvectorize([(float32, float32, float32[:], float32[:]),
              (float64, float64, float64[:], float64[:])],'(),(),(m)->(m)', nopython=True, fastmath=True, *args, **kwargs_)(func)

def random_normal_bias(mu, sigma, arr, out):
    dim1 = arr.shape
    for i in prange(*dim1):
            out[i] = fabs(normalvariate(mu, sigma))

In [17]:
weights = np.zeros((20000,20000), dtype=np.float32)
bias = np.zeros((20000), dtype=np.float32)
s_rnd_w = gu_random_normal_weights(random_normal_weights)
p_rnd_w = gu_random_normal_weights(random_normal_weights, target='parallel')
s_rnd_b = gu_random_normal_bias(random_normal_bias)
p_rnd_b = gu_random_normal_bias(random_normal_bias, target='parallel')

In [18]:

def ntime(func1):
    start = time.time()
    func1(0, 2, weights)
    end = time.time()
    print("Elapsed Numba Pre = %s" % (end - start))
    start = time.time()
    f2 = func1(0, 2, weights)
    end = time.time()
    print("Elapsed Numba Post = %s" % (end - start))
    return f2

def ptime():
    start = time.time()
    f1 = np.random.normal(0, 2, (20000, 20000))
    # f1 = np.abs(np.random.normal(0, 2, (200000)))
    end = time.time()
    print("Elapsed PyTime = %s" % (end - start))
    return f1

Elapsed Numba Pre = 7.467274904251099
Elapsed Numba Post = 7.365271091461182


array([[ 0.37455055, -0.24131782,  1.5403347 , ..., -1.0425339 ,
        -0.8213711 ,  2.7395089 ],
       [ 4.459771  ,  0.6702843 , -2.1552598 , ...,  2.9178855 ,
         2.3215299 , -0.06290729],
       [-0.2020432 , -1.1531245 , -1.2291946 , ..., -4.994263  ,
        -0.8146394 ,  4.189339  ],
       ...,
       [-2.3413703 , -2.775388  , -1.5667449 , ...,  1.8865016 ,
         1.3237101 , -0.23246406],
       [-1.3383389 , -3.8944755 , -4.263738  , ...,  1.2506982 ,
         2.5915637 , -0.08051662],
       [-0.22997254, -3.6572802 , -1.5947757 , ...,  1.9915048 ,
         2.0837495 , -0.03423833]], dtype=float32)